In [1]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from collections import Counter

from keras.preprocessing import sequence
from keras.models import load_model, Model
from keras.layers import Dense, Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
def create_dictionary(texts, vocab_size):
    """
    Creates a dictionary that maps words to ids. More frequent words have lower ids.
    The dictionary contains at the vocab_size-1 most frequent words (and a placeholder '<unk>' for unknown words).
    The place holder has the id 0.
    """
    counter = Counter()
    for tokens in texts:
        counter.update(tokens)
    vocab = [w for w, c in counter.most_common(vocab_size - 1)]
    word_to_id = {w: (i + 1) for i, w in enumerate(vocab)}
    word_to_id[UNKNOWN_TOKEN] = 0
    return word_to_id


def to_ids(words, dictionary):
    """
    Takes a list of words and converts them to ids using the word2id dictionary.
    """
    ids = []
    for word in words:
        ids.append(dictionary.get(word, dictionary[UNKNOWN_TOKEN]))
    return ids


def read_data(train_file, dev_file):
    """
        Liest alle Daten in ein Pandas Dataframe ein. Da noch keine Testdaten vorliegen werden aus den zufällig
        gemischten Tweets Train, Dev und Testset erstellt (80/10/10 Split).
        Die Tweets werden per NLTK Tweet Tokenizer gesplittet.
    """

    trainDF = pd.read_csv(train_file, sep='\t')
    devDF = pd.read_csv(dev_file, sep='\t')

    allDF = pd.concat([trainDF, devDF], ignore_index=True)
    allDF = allDF.reindex(np.random.permutation(allDF.index))

    #Einfügen der tokenisierten Version des Textes:
    tokenizer = TweetTokenizer()
    allDF.insert(1, 'tweet_tokenized', (allDF['Tweet'].apply(lambda x: tokenizer.tokenize(x))))

    # Einfügen der WordIDs:
    word2id = create_dictionary(allDF["tweet_tokenized"], VOCAB_SIZE)
    allDF.insert(1, 'tweet_ids', (allDF['Tweet'].apply(lambda x: to_ids(x, dictionary=word2id))))

    # Konkatenieren der binären Emotionen (die in den letzten 11 Spalten des Dataframes stehen) zu einer Liste:
    allDF['all'] = allDF.iloc[:, -11:].values.tolist()

    # Aufteilen in 3 Sets:
    total = len(allDF)
    trainend = int(total * 0.8)
    devend = trainend + int(total * 0.1)
    return allDF.iloc[:trainend, :], allDF.iloc[trainend:devend, :], allDF.iloc[devend:, :]


def evaluate(predictions, y_test):
    """
        Funktion zur Berechnung von Precision, Recall und F1 Score. 
    """
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for i, pred in enumerate(predictions):
        for j, em in enumerate(pred):
            if em >= 0.5:
                if y_test[i][j] == 1:
                    tp += 1
                else:
                    fp += 1
            if em < 0.5:
                if y_test[i][j] == 1:
                    fn += 1
                else:
                    tn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    print("F1: {}\nPrecision: {}\nRecall: {}".format(f1, precision, recall))


Setzen der globalen Variablen


In [3]:
data_dir = 'data/'
train_file = os.path.join(data_dir, '2018-E-c-En-train.txt')
dev_file = os.path.join(data_dir, '2018-E-c-En-dev.txt')

VOCAB_SIZE = 100000  # Aufgrund der Kürze der Tweets: Große Vocab Size, damit der Anteil der unknown words nicht zu hoch ist
MAX_LEN = 100
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 50
EPOCHS = 50 # Hohe Anzahl, die wegen Early Stopping nicht erreicht werden sollte
UNKNOWN_TOKEN = "<unk>"

Einlesen der Daten in Training-, Development-, und Testset

Die Daten werden in Pandas-Dataframes gespeichert, ursprünglich um auch die Daten der Emotionen einzeln verwenden zu können.

x_train, x_dev, x_test bestehen aus den Inputs als Word-ID-Vektoren.

y_train, y_dev, y_test bestehen aus den Output-Vektoren, mit jeweils 0 oder 1 für die einzelnen Emotionen.



In [4]:
trainDF, devDF, testDF = read_data(train_file, dev_file)

x_train = sequence.pad_sequences(np.array(trainDF['tweet_ids']), maxlen=MAX_LEN)
x_dev = sequence.pad_sequences(np.array(devDF['tweet_ids']), maxlen=MAX_LEN)
x_test = sequence.pad_sequences(np.array(testDF['tweet_ids']), maxlen=MAX_LEN)

y_train = np.array([trainDF['all']])[0]
y_dev = np.array([devDF['all']])[0]
y_test = np.array([testDF['all']])[0]


Aufbau des Neuronalen Netzes in der Keras Functional API

Getestet wurde auch ein bidirektionales LSTM, allerdings haben sich dort die Ergebnisse nicht verbessert und die Laufzeit stieg um das 30 bis 60-fache.

Um das Testen der Hyperparameter (zeitlich) zu erleichtern wurde das CNN weiter verwendet, auch weil es sehr gute Ergebnisse erzielte.


In [5]:
main_input = Input(shape=(MAX_LEN,), dtype='int32', name='main_input')

x = Embedding(VOCAB_SIZE, EMBEDDING_SIZE)(main_input)
x = Conv1D(
    filters=2 * HIDDEN_SIZE,
    kernel_size=3,
    activation='tanh',
    strides=1,
    padding='valid', )(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.5)(x)

predictions = Dense(11, activation='sigmoid')(x)



In [6]:
model = Model(inputs=main_input, outputs=predictions)


Kompilieren des Modells

Hier wurden unterschiedliche Optimizer getestet (u.a. auch ein 'adam' Optimizer mit unterschiedlichen Lernraten).

Am besten performt hier rmsprop.

In [7]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              )


Implementierung von Early Stopping und Checkpoint nach Vorbild der in der Vorlesung präsentierten Folien.


In [8]:
early_stopper = EarlyStopping(monitor='val_acc', patience=3, mode='max')
checkpoint = ModelCheckpoint(data_dir + 'model.m', save_best_only=True, monitor='val_acc', mode='max')

In [9]:
def compute_class_weights(y_train):
    """
    Berechnet Class Weights. Klassen die seltener vorkommen werden höher gewichtet.
    Genauere Analyse des Einflusses folgt im Kurzvortrag, bzw. der Ausarbeitung.
    """
    class_totals = np.sum(y_train, axis=0)
    overall_sum = np.sum(y_train)
    class_weights = [round(((overall_sum - i) / overall_sum) ** 5, 4) for i in class_totals]
    return class_weights


class_weights = compute_class_weights(y_train)

In [10]:
model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopper, checkpoint],
    epochs=EPOCHS,
    validation_data=(x_dev, y_dev),
    class_weight=class_weights,
    verbose=1
)

Train on 6179 samples, validate on 772 samples
Epoch 1/50


  32/6179 [..............................] - ETA: 6:39 - loss: 0.6705 - acc: 0.6477

  96/6179 [..............................] - ETA: 2:15 - loss: 0.6507 - acc: 0.7083

 160/6179 [..............................] - ETA: 1:22 - loss: 0.6339 - acc: 0.7381

 224/6179 [>.............................] - ETA: 1:00 - loss: 0.6210 - acc: 0.7463

...

5984/6179 [============================>.] - ETA: 0s - loss: 0.3800 - acc: 0.8372

6048/6179 [============================>.] - ETA: 0s - loss: 0.3799 - acc: 0.8372

6112/6179 [============================>.] - ETA: 0s - loss: 0.3801 - acc: 0.8370

6176/6179 [============================>.] - ETA: 0s - loss: 0.3801 - acc: 0.8370

6179/6179 [==============================] - 7s 1ms/step - loss: 0.3802 - acc: 0.8370 - val_loss: 0.3973 - val_acc: 0.8291


In [11]:
best_model = load_model(data_dir+'model.m')

In [12]:
predictions = best_model.predict(x_test)

Aufruf der Evaluate Funktion, die F1 Score, Precision und Recall berechnet.


In [ ]:
evaluate(predictions, y_test)